In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import io
import chardet
import os
import glob

In [ ]:
# 自动识别当前文件夹下唯一的csv文件
csv_files = glob.glob('*.csv')
if len(csv_files) == 1:
    file_path = csv_files[0]
else:
    raise FileNotFoundError('未找到唯一的csv文件，请检查文件夹内容')


In [ ]:
# Detect file encoding
with open(file_path, 'rb') as f:
    rawdata = f.read(10000)  # Only read first 10,000 bytes for encoding detection
    result = chardet.detect(rawdata)
    encoding = result['encoding']
    print(f"Detected file encoding: {encoding}")

# Read file content
try:
    with open(file_path, 'r', encoding=encoding) as f:
        content = f.read()
except:
    # If detected encoding fails, try utf-8 with error ignoring
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

# Replace possible null characters
content = content.replace('\x00', '')

# Read content into DataFrame
df = pd.read_csv(io.StringIO(content), sep='\t')


In [ ]:
print("First five lines:")
print(df.head())

df = df.reset_index(drop=True)


In [ ]:
# 1. 识别数据块结构
# 每个测量指标占据两列：第1列为体积(ml)，第2列为测量值

# 2. 提取各数据系列
data_series = {
    'UV_280': {'vol_col': 0, 'data_col': 1, 'unit': 'mAU'},
    'Cond': {'vol_col': 2, 'data_col': 3, 'unit': 'mS/cm'},
    'Conc_B': {'vol_col': 4, 'data_col': 5, 'unit': '%'},
    # 'UV_280_CUT': {'vol_col': 8, 'data_col': 9, 'unit': 'mAU'}
}

# 3. 创建新的数据框
processed_data = pd.DataFrame()

# 4. 处理每个数据系列
for name, info in data_series.items():
    # 提取体积列(跳过前两行标题)
    vol = pd.to_numeric(df.iloc[2:, info['vol_col']], errors='coerce')
    # 提取数据列
    values = pd.to_numeric(df.iloc[2:, info['data_col']], errors='coerce')
    
    # 添加到新数据框
    processed_data[f'{name}_Volume'] = vol
    processed_data[name] = values

# 5. 清理数据 - 移除全为NaN的行
processed_data = processed_data.dropna(how='all')

# 重置索引
processed_data = processed_data.reset_index(drop=True)

# 计算UV_280 = UV_280_Volume - UV_280_CUT_Volume
# processed_data['UV_280'] = processed_data['UV_280'] - processed_data['UV_280_CUT']

# # 筛选数据：只保留UV_280_Volume >= 60mL的数据点
# # 但保留所有列的数据
# filter_condition = processed_data['UV_280_Volume'] >= 60
# processed_data = processed_data[filter_condition].copy()

print("processed data: \n")
print(processed_data.head())

In [ ]:
# 提取 Fraction 体积（第6列）和标签（第7列）
fraction_volumes = pd.to_numeric(df.iloc[2:, 6], errors='coerce')
fraction_labels = df.iloc[2:, 7]

# 创建 Fraction 数据框
fraction_df = pd.DataFrame({
    'Frac_Volume': fraction_volumes,
    'Frac': fraction_labels
})

# 清理数据：移除空标签行
fraction_df = fraction_df.dropna(subset=['Frac'])
fraction_df = fraction_df[fraction_df['Frac'] != '']

# 重置索引
fraction_df = fraction_df.reset_index(drop=True)

print("fraction_df:")
fraction_df

In [ ]:
processed_data

In [ ]:
plt.rcParams.update({
    'font.family': 'Times New Roman',    # Change font type
    'font.weight': 'bold',     
    'axes.labelweight': 'bold',   # Bold font
    'font.size': 20,           # Default font size
    'lines.linewidth': 3.0,     # Increase line width
    # 'axes.titlesize': 20,      # Title font size
    # 'axes.labelsize': 20,      # Axis label font size
    # 'xtick.labelsize': 16,     # X-axis tick label size
    # 'ytick.labelsize': 16,     # Y-axis tick label size
    # 'legend.fontsize': 24      # Legend font size
})

# Create figure and left axis
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot UV 280 (left Y-axis)
color = 'blue'
ax1.set_xlabel('Volume (mL)')
ax1.set_ylabel('UV 280 (mAU)', color='black')
ax1.plot(processed_data['UV_280_Volume'], processed_data['UV_280'], 
         color=color, label='UV 280 (mAU)')
ax1.tick_params(axis='y', labelcolor='black')

# Create right axis
ax2 = ax1.twinx()
color = 'red'
ax2.set_ylabel('Conc B (%)', color='black')
ax2.plot(processed_data['Conc_B_Volume'], processed_data['Conc_B'], 
         color=color, label='Conc B (%)')
ax2.tick_params(axis='y', labelcolor='black')

# 添加 Fraction 收集点标记
# 1. 在x轴下方添加小的垂直线标记
for vol in fraction_df['Frac_Volume']:
    # 在x轴下方添加短垂直线（位置在y=0上方）
    ax1.axvline(x=vol, color='darkgreen', linestyle='-', alpha=0.8, ymin=0.005, ymax=0.01, clip_on=False)

# 2. 添加Fraction标签（旋转90度，放在x轴下方）
for idx, row in fraction_df.iterrows():
    vol = row['Frac_Volume']
    label = row['Frac']

    # 只标记每个组的第一个收集点（如5.A.1, 5.B.1等），避免标签重叠
    if label.endswith('A.1') or label == 'Waste(Frac)':
        ax1.text(vol, 0.05, label,
                 transform=ax1.get_xaxis_transform(),  # 使用x轴变换坐标
                 rotation=0,
                 verticalalignment='top',
                 horizontalalignment='center',
                 fontsize=10,
                 color='darkgreen',
                 bbox=dict(facecolor='white', alpha=0, pad=1, edgecolor='none', boxstyle='round'))

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper center', 
           framealpha=0.5, frameon=True, edgecolor='black')

# Configure grid lines (only horizontal)
ax1.grid(axis='y', linestyle='--', alpha=0.7)
ax1.grid(axis='x', visible=False)  # Hide vertical grid lines

# Set axis range
ax1.set_xlim(left=0)
ax1.set_xlim(right=30)
ax1.set_ylim(bottom=0)
ax1.set_ylim(top=200)
ax2.set_ylim(bottom=0)
ax2.set_ylim(top=100)

# Save the plot
plt.savefig(file_path.replace('.csv', '') + '.jpg', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# Create figure and left axis
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot UV 280 (left Y-axis)
color = 'blue'
ax1.set_xlabel('Volume (mL)')
ax1.set_ylabel('UV 280 (mAU)', color='black')
ax1.plot(processed_data['UV_280_Volume'], processed_data['UV_280'],
         color=color, label='UV 280 (mAU)')
ax1.tick_params(axis='y', labelcolor='black')

# Create right axis
ax2 = ax1.twinx()
color = 'red'
ax2.set_ylabel('Conc B (%)', color='black')
ax2.plot(processed_data['Conc_B_Volume'], processed_data['Conc_B'],
         color=color, label='Conc B (%)')
ax2.tick_params(axis='y', labelcolor='black')

# 添加 Fraction 收集点标记
# 1. 在x轴下方添加小的垂直线标记
for vol in fraction_df['Frac_Volume']:
    # 在x轴下方添加短垂直线（位置在y=0上方）
    ax1.axvline(x=vol, color='darkgreen', linestyle='-', alpha=0.8, ymin=0.005, ymax=0.01, clip_on=False)

# 2. 添加Fraction标签（旋转90度，放在x轴下方）
for idx, row in fraction_df.iterrows():
    vol = row['Frac_Volume']
    label = row['Frac']

    # 只标记每个组的第一个收集点（如5.A.1, 5.B.1等），避免标签重叠
    if label.endswith('.3') or label.endswith('.8'):
        ax1.text(vol, 0.05, label,
                 transform=ax1.get_xaxis_transform(),  # 使用x轴变换坐标
                 rotation=0,
                 verticalalignment='top',
                 horizontalalignment='center',
                 fontsize=10,
                 color='darkgreen',
                 bbox=dict(facecolor='white', alpha=0, pad=1, edgecolor='none', boxstyle='round'))

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper center',
           framealpha=0.5, frameon=True, edgecolor='black')

# Configure grid lines (only horizontal)
ax1.grid(axis='y', linestyle='--', alpha=0.7)
ax1.grid(axis='x', visible=False)  # Hide vertical grid lines

# Set axis range
ax1.set_xlim(left=0)
ax1.set_xlim(right=30)
ax1.set_ylim(bottom=0)
ax1.set_ylim(top=200)
ax2.set_ylim(bottom=0)
ax2.set_ylim(top=100)

plt.savefig(file_path.replace('.csv', '') + '_Fracs.jpg', dpi=1600, bbox_inches='tight')
plt.show()